Model trained on contaxts being original_code. 

In [1]:
from transformers import (
    AutoTokenizer,
    TFAutoModelForQuestionAnswering,
    DataCollatorForSeq2Seq,
    keras_callbacks,
    TFAutoModelForSeq2SeqLM,
    TFEncoderDecoderModel,
)
import tensorflow as tf
from huggingface_hub import notebook_login
from question_answering.constants import constants
from question_answering.utils import core_qa_utils, generative_qa_utils
from question_answering.paths import generative_qa_paths

In [2]:
df_train, df_val, df_test = core_qa_utils.load_train_val_test_datasets(
    generative_qa_paths.python_dataset_dir
)

train_dataset, val_dataset, test_dataset = core_qa_utils.convert_dataframes_to_datasets(
    [df_train, df_val, df_test]
)

In [3]:
df_train

,index,questions,answers,code,original_code
0,0,What does the code make ?,a suite,def Make Suite From Dict d label None suite Su...,def MakeSuiteFromDict d label None suite Suite...
1,1,Does the code make a suite ?,Yes,def Make Suite From Dict d label None suite Su...,def MakeSuiteFromDict d label None suite Suite...
2,2,Does the code receive a message from a pull su...,Yes,def receive message topic name subscription na...,def receive_message topic_name subscription_na...
3,3,What does the code receive from a pull subscri...,a message,def receive message topic name subscription na...,def receive_message topic_name subscription_na...
4,4,What be an explicit budget used only ?,to create the campaign,def Create Shared Budget client budget service...,def CreateSharedBudget client budget_service c...
...,...,...,...,...,...
56080,56080,What compiles the file filename ?,bytecode,def save pyc filename cfile '%sc' % filename p...,def save_pyc filename cfile '%sc' % filename p...
56081,56081,What will this function spawn ?,a thread,def with timeout func args kwargs {} class Res...,def with_timeout func args kwargs {} class Res...
56082,56082,What do this function run using the args ?,the given function,def with timeout func args kwargs {} class Res...,def with_timeout func args kwargs {} class Res...
56083,56083,How do this function run the given function ?,using the args,def with timeout func args kwargs {} class Res...,def with_timeout func args kwargs {} class Res...


In [4]:
# model_checkpoint = "t5-small"
model_checkpoint = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
def tokenize_sample(sample, max_tokens=None, padding=False):
    question = sample["questions"].strip()
    context = sample["original_code"].strip()

    question_context = context + ' - ' + question

    return tokenizer(question, context, max_length=max_tokens, padding=padding)
    # return tokenizer(question_context, max_length=max_tokens, padding=padding)


tokenized_train_dataset = train_dataset.map(tokenize_sample)
tokenized_val_dataset = val_dataset.map(tokenize_sample)
tokenized_test_dataset = test_dataset.map(tokenize_sample)

print(
    "Max number of tokens in tokenized train dataset: ",
    len(max(tokenized_train_dataset["input_ids"], key=len)),
)
print(
    "Max number of tokens in tokenized val dataset: ",
    len(max(tokenized_val_dataset["input_ids"], key=len)),
)
print(
    "Max number of tokens in tokenized test dataset: ",
    len(max(tokenized_test_dataset["input_ids"], key=len)),
)

Map:   0%|          | 0/56080 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1490 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Max number of tokens in tokenized train dataset:  9244
Max number of tokens in tokenized val dataset:  575
Max number of tokens in tokenized test dataset:  901


In [6]:
max_length = 256


def filter_samples_below_number_of_tokens(dataset, max_tokens: int):
    indices_to_remove = []

    # Find indices of samples where number of tokens exceeds max number of tokens
    for index, sample in enumerate(dataset):
        tokenized_sample = tokenize_sample(sample)
        if len(tokenized_sample["input_ids"]) > max_tokens:
            indices_to_remove.append(index)

    # Keep only samples with number of tokens less or equal than max number of tokens
    dataset_indices = range(len(dataset))
    filtered_dataset = dataset.select(
        index for index in dataset_indices if index not in set(indices_to_remove)
    )

    return filtered_dataset


filtered_train_dataset = filter_samples_below_number_of_tokens(
    train_dataset, max_tokens=max_length
)
filtered_val_dataset = filter_samples_below_number_of_tokens(
    val_dataset, max_tokens=max_length
)
filtered_test_dataset = filter_samples_below_number_of_tokens(
    test_dataset, max_tokens=max_length
)

Parameter 'indices'=<generator object filter_samples_below_number_of_tokens.<locals>.<genexpr> at 0x000001B6D228B610> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [7]:
print(
    "Number of samples in tokenized train dataset before filtering: ",
    len(train_dataset),
)
print("Number of samples in tokenized val dataset before filtering: ", len(val_dataset))
print(
    "Number of samples in tokenized test dataset before filtering: ", len(test_dataset)
)

print("\n---------------\n")

print(
    "Number of samples in tokenized train dataset after filtering: ",
    len(filtered_train_dataset),
)
print(
    "Number of samples in tokenized val dataset after filtering: ",
    len(filtered_val_dataset),
)
print(
    "Number of samples in tokenized test dataset after filtering: ",
    len(filtered_test_dataset),
)

Number of samples in tokenized train dataset before filtering:  56080
Number of samples in tokenized val dataset before filtering:  7000
Number of samples in tokenized test dataset before filtering:  7000

---------------

Number of samples in tokenized train dataset after filtering:  54930
Number of samples in tokenized val dataset after filtering:  6828
Number of samples in tokenized test dataset after filtering:  6854


In [8]:
# def combine_question_and_context(dataset_row):
#     context_question = dataset_row['original_code'] + ' - ' + dataset_row['questions']
#     dataset_row['context_question'] = context_question
#     return dataset_row

In [9]:
# filtered_train_dataset = filtered_train_dataset.map(combine_question_and_context)
# filtered_val_dataset = filtered_val_dataset.map(combine_question_and_context)
# filtered_test_dataset = filtered_test_dataset.map(combine_question_and_context)

In [10]:
# filtered_train_dataset['context_question'][0]

In [11]:
def preprocess_dataset(dataset):
    questions = [q.strip() for q in dataset["questions"]]
    contexts = [c.strip() for c in dataset["original_code"]]
    answers = [c.strip() for c in dataset["answers"]]

    # question_context = [cq.strip() for cq in dataset["context_question"]]

    inputs = tokenizer(
        questions,
        contexts,
        # question_context,
        text_target=answers,
        max_length=max_length,
        padding="max_length",
        return_offsets_mapping=True,
    )

    return inputs

In [12]:
tokenized_train_dataset = filtered_train_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_train_dataset.column_names,
)
tokenized_val_dataset = filtered_val_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_val_dataset.column_names,
)
tokenized_test_dataset = filtered_test_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_test_dataset.column_names,
)

Map:   0%|          | 0/54930 [00:00<?, ? examples/s]

Map:   0%|          | 0/6828 [00:00<?, ? examples/s]

Map:   0%|          | 0/6854 [00:00<?, ? examples/s]

In [13]:
print(
    f"All tokenized train dataset entries have {max_length} tokens: ",
    all(
        [
            len(input_ids) == max_length
            for input_ids in tokenized_train_dataset["input_ids"]
        ]
    ),
)
print(
    f"All tokenized val dataset entries have {max_length} tokens: ",
    all(
        [
            len(input_ids) == max_length
            for input_ids in tokenized_val_dataset["input_ids"]
        ]
    ),
)
print(
    f"All tokenized test dataset entries have {max_length} tokens: ",
    all(
        [
            len(input_ids) == max_length
            for input_ids in tokenized_test_dataset["input_ids"]
        ]
    ),
)

All tokenized train dataset entries have 256 tokens:  True
All tokenized val dataset entries have 256 tokens:  True
All tokenized test dataset entries have 256 tokens:  True


In [14]:
# Model parameters
training_number = 5

model_name = "python-bart-uncased"
full_model_name = f"{model_name}-{training_number}"

# Checkpoints
checkpoint_filename_template = constants.checkpoint_filename_template
checkpoints_path = (
    generative_qa_paths.training_checkpoints_dir
    / full_model_name
    / checkpoint_filename_template
)

# Hub
hub_path = generative_qa_paths.hub_models_location / full_model_name

# Saved models
saved_models_path = generative_qa_paths.saved_models_dir / full_model_name

# Figures
figures_dir = generative_qa_paths.figures_dir / full_model_name

# Hyperparameters
batch_size = 8
train_epochs = 1

In [15]:
# Load model for fine-tuning
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForConditionalGeneration: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [16]:
# Dataset preparation
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

tf_train_dataset = core_qa_utils.prepare_tf_dataset(
    model=model,
    hf_dataset=tokenized_train_dataset,
    collator=data_collator,
    batch_size=batch_size,
    shuffle=True
)

tf_val_dataset = core_qa_utils.prepare_tf_dataset(
    model=model,
    hf_dataset=tokenized_val_dataset,
    collator=data_collator,
    batch_size=batch_size,
)

tf_test_dataset = core_qa_utils.prepare_tf_dataset(
    model=model,
    hf_dataset=tokenized_test_dataset,
    collator=data_collator,
    batch_size=batch_size,
)

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [17]:
# Callbacks
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    checkpoints_path, verbose=1, save_weights_only=True
)
early_stop_cb = tf.keras.callbacks.EarlyStopping(patience=1)
# push_to_hub = keras_callbacks.PushToHubCallback(
#     output_dir=full_model_name, tokenizer=tokenizer
# )

callbacks = [
    checkpoint_cb,
    early_stop_cb,
    # push_to_hub
]

In [18]:
# Compile
num_train_steps = len(tf_train_dataset) * train_epochs

lr_scheduler = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# Compile
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metrics = ["accuracy"]
# model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
# model.compile(optimizer=optimizer, metrics=metrics)
model.compile(optimizer=optimizer)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070, compute capability 8.9


In [19]:
model.summary()

Model: "tf_bart_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBartMainLayer)     multiple                  139420416 
                                                                 
 final_logits_bias (BiasLaye  multiple                 50265     
 r)                                                              
                                                                 
Total params: 139,470,681
Trainable params: 139,420,416
Non-trainable params: 50,265
_________________________________________________________________


In [20]:
# Fit the model on the new data
history = model.fit(
    tf_train_dataset,
    validation_data=tf_val_dataset,
    epochs=train_epochs,
    callbacks=callbacks,
)

6866/6866 [==============================] - ETA: 0s - loss: 2.6914
Epoch 1: saving model to e:\STUDIA\IPS\question-answering\generative-qa\training-checkpoints\python-bart-uncased-5\cp-01.ckpt
6866/6866 [==============================] - 1500s 216ms/step - loss: 2.6914 - val_loss: 2.7224


In [21]:
# Get best version of the model
best_model, best_epoch = core_qa_utils.get_best_model_from_checkpoints(
    model, history, model_name=full_model_name, remove_checkpoints=True, model_type="generative"
)

In [22]:
# Save best model's weights
generative_qa_utils.save_model(best_model, model_name=full_model_name)

In [23]:
full_model_name

'python-bart-uncased-5'

In [36]:
# Load best model
loaded_model = generative_qa_utils.load_model(
    model_checkpoint, model_name=full_model_name
)
loaded_model.compile(optimizer=optimizer)

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [64]:
loaded_weights_model = generative_qa_utils.load_weights_into_model(
    model=model, 
    model_name=full_model_name
)

In [22]:
# Get predictions from the best model
loaded_model_evaluation = loaded_model.evaluate(tf_test_dataset)

857/857 [==============================] - 74s 73ms/step - loss: 2.6827


In [34]:
import evaluate

metric = evaluate.load("bleu")

In [65]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

def generate_with_xla(batch):
    return loaded_weights_model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=max_length,
    )


In [25]:
tokenized_test_dataset2 = tokenized_test_dataset.train_test_split(test_size=0.001)['test']
len(tokenized_test_dataset2[0]['input_ids'])

256

In [69]:
tokenizer.decode(tokenized_test_dataset[0]['input_ids'])

"<s>How does the code add a user in the given buckets default object access control list?</s></s>def add_bucket_default_owner bucket_name user_email storage_client storage Client bucket storage_client bucket bucket_name bucket acl reload bucket default_object_acl user user_email grant_owner bucket default_object_acl save print 'Addeduser{}asanownerinthedefaultaclonbucket{}'format user_email bucket_name</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [26]:
tokenizer.decode(tokenized_test_dataset2[0]['input_ids'])

'<s>What will they have?</s></s>def add_course_author user course global_admin AdminFactory for role in CourseStaffRole CourseInstructorRole auth add_users global_admin role course id user</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [27]:
tokenizer.decode(tokenized_test_dataset2[0]['input_ids'])

'<s>What will they have?</s></s>def add_course_author user course global_admin AdminFactory for role in CourseStaffRole CourseInstructorRole auth add_users global_admin role course id user</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [28]:
tokenizer.decode(tokenized_test_dataset2[0]['labels'])

'<s>the permissions to see it in studio</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [47]:
loaded_model.summary()

Model: "tf_bart_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBartMainLayer)     multiple                  139420416 
                                                                 
 final_logits_bias (BiasLaye  multiple                 50265     
 r)                                                              
                                                                 
Total params: 139,470,681
Trainable params: 139,420,416
Non-trainable params: 50,265
_________________________________________________________________


In [48]:
best_model.summary()

Model: "tf_bart_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBartMainLayer)     multiple                  139420416 
                                                                 
 final_logits_bias (BiasLaye  multiple                 50265     
 r)                                                              
                                                                 
Total params: 139,470,681
Trainable params: 139,420,416
Non-trainable params: 50,265
_________________________________________________________________


In [66]:
tf_test_dataset2 = core_qa_utils.prepare_tf_dataset(
    model=loaded_weights_model,
    hf_dataset=tokenized_test_dataset2,
    collator=data_collator,
    batch_size=batch_size,
)

In [57]:
tf_test_dataset2

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 256), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 256), dtype=tf.int64, name=None), 'decoder_input_ids': TensorSpec(shape=(None, 256), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 256), dtype=tf.int64, name=None))>

In [54]:
tf_test_dataset2

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 256), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 256), dtype=tf.int64, name=None), 'decoder_input_ids': TensorSpec(shape=(None, 256), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 256), dtype=tf.int64, name=None))>

In [136]:
import pandas as pd

all_preds = []
all_labels = []
all_labels2 = []
something = []
something2 = []
df = pd.DataFrame()

for batch, labels in tqdm(tf_test_dataset2):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)
    all_labels2.extend(tokenizer.batch_decode(labels, skip_special_tokens=True))
    something.extend(tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True))
    something2.extend(tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True))
    data = {
        'qc': tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True),
        'labels': decoded_labels,
        'preds': decoded_preds
    }
    df = pd.DataFrame(data)

100%|██████████| 1/1 [00:05<00:00,  5.13s/it]


In [102]:
all_preds, all_labels, all_labels2, something, something2

(['access to the course',
  'with the given part',
  'the event loop',
  'the hamming loss',
  'a kaiserformula',
  'a function',
  'with the given authz'],
 [['the permissions to see it in studio'],
  ['in the form of a key / value pair separated by a colon'],
  ['the qt4 event loop'],
  ['the average hamming loss'],
  ['a kaiser window'],
  ['skip if a test is executed under travis testing environment'],
  ['with their translations']],
 ['the permissions to see it in studio',
  'in the form of a key / value pair separated by a colon',
  'the qt4 event loop',
  'the average hamming loss',
  'a kaiser window',
  'skip if a test is executed under travis testing environment',
  'with their translations'],
 ['What will they have?def add_course_author user course global_admin AdminFactory for role in CourseStaffRole CourseInstructorRole auth add_users global_admin role course id user',
  "How does the code take a query?def parse_query_part part query_classes {} prefixes {} default_class qu

In [114]:
df

,qc,labels,preds
0,What will they have?def add_course_author user...,[the permissions to see it in studio],access to the course
1,How does the code take a query?def parse_query...,[in the form of a key / value pair separated b...,with the given part
2,What does the code start in a consistent manne...,[the qt4 event loop],the event loop
3,What does the code compute?def hamming_loss y_...,[the average hamming loss],the hamming loss
4,What limits ripple and width of transition reg...,[a kaiser window],a kaiserformula
5,What could be used as a decorator or unparamet...,[skip if a test is executed under travis testi...,a function
6,How do a dict of available roles return?@core_...,[with their translations],with the given authz


In [139]:
questions = []
contexts = []

for index, row in df.iterrows():
    questions.append(row['qc'].split('?')[0] + '?')
    contexts.append(row['qc'].split('?')[1])

df['questions'] = questions
df['contexts'] = contexts
labels = df['labels']
preds = df['preds']
df = df.drop(['qc', 'labels', 'preds'], axis=1)
df['labels'] = labels
df['preds'] = preds

KeyError: 'qc'

In [140]:
df

,questions,contexts,labels,preds
0,What will they have?,def add_course_author user course global_admin...,[the permissions to see it in studio],access to the course
1,How does the code take a query?,def parse_query_part part query_classes {} pre...,[in the form of a key / value pair separated b...,with the given part
2,What does the code start in a consistent manner?,def start_event_loop_qt4 app None if app is No...,[the qt4 event loop],the event loop
3,What does the code compute?,def hamming_loss y_true y_pred labels None sam...,[the average hamming loss],the hamming loss
4,What limits ripple and width of transition reg...,def kaiserord ripple width A abs ripple if A <...,[a kaiser window],a kaiserformula
5,What could be used as a decorator or unparamet...,def skip_under_travis fn None if _travisTestin...,[skip if a test is executed under travis testi...,a function
6,How do a dict of available roles return?,@core_helperdef roles_translated return authz ...,[with their translations],with the given authz


In [156]:
df.sample(n = 1)

,questions,contexts,labels,preds
5,What could be used as a decorator or unparamet...,def skip_under_travis fn None if _travisTestin...,[skip if a test is executed under travis testi...,a function


In [35]:
result = metric.compute(predictions=all_preds, references=all_labels)
result

{'bleu': 0.0,
 'precisions': [0.5454545454545454, 0.13333333333333333, 0.0, 0.0],
 'brevity_penalty': 0.36787944117144233,
 'length_ratio': 0.5,
 'translation_length': 22,
 'reference_length': 44}